In [ ]:
import mne
import scipy
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import utils
import algo
import os
import random
import glob
import copy
import itertools
import pickle
from findpeaks import findpeaks
from tqdm import tqdm
from numpy import linalg as LA
from scipy.stats import zscore, pearsonr
from scipy.io import savemat, loadmat
from scipy import signal
%matplotlib widget

## Data Loading

In [ ]:
def multisub_data_org(subjects, video_id, fsStim, bads, band=None, eog=False, regression=False, normalize=True, smooth=True):
    feats_path_folder = '../Feature extraction/features/'
    objf = np.load(feats_path_folder + video_id + '_mask.npy')
    flow = np.load(feats_path_folder + video_id + '_flow.npy')
    flow_mag = np.expand_dims(flow[:,8], axis=1)
    tempctra = np.load(feats_path_folder + video_id + '_tempctra.npy')
    tc_mask = np.load(feats_path_folder + video_id + '_tcmask.npy')
    feats = np.concatenate((objf, tc_mask, flow_mag, tempctra), axis=1)
    feats = utils.clean_features(feats, smooth=smooth)
    T = feats.shape[0]
    eeg_list = []
    hf_list = []
    eog_list = []
    N = len(subjects)
    for n in range(N):
        eeg_path = '../../Experiments/data/'+ subjects[n] +'/' + video_id + '_eeg.set'
        eeg_prepro, fs, high_freq = utils.preprocessing(eeg_path, HP_cutoff = 0.5, AC_freqs=50, band=band, resamp_freqs=fsStim, bads=bads[n], eog=eog, regression=regression, normalize=normalize)
        eeg_channel_indices = mne.pick_types(eeg_prepro.info, eeg=True)
        eog_channel_indices = mne.pick_types(eeg_prepro.info, eog=True)
        eeg_downsampled, _ = eeg_prepro[eeg_channel_indices]
        eog_downsampled, _ = eeg_prepro[eog_channel_indices]
        eeg_downsampled = eeg_downsampled.T
        eog_downsampled = eog_downsampled.T
        eeg_list.append(eeg_downsampled)
        eog_list.append(eog_downsampled)
        if eeg_downsampled.shape[0] < T:
            T = eeg_downsampled.shape[0]
        hf_indices = mne.pick_types(high_freq.info, eeg=True)
        hf_downsampled, _ = high_freq[hf_indices]
        hf_downsampled = hf_downsampled.T
        hf_list.append(hf_downsampled)
        if hf_downsampled.shape[0] < T:
            T = hf_downsampled.shape[0] 
    # Clip data
    feats = feats[fsStim:T-fsStim, :]
    eeg_list = [np.expand_dims(eeg[fsStim:T-fsStim,:], axis=2) for eeg in eeg_list]
    eog_list = [np.expand_dims(eog[fsStim:T-fsStim,:], axis=2) for eog in eog_list]
    hf_list = [np.expand_dims(eeg[fsStim:T-fsStim,:], axis=2) for eeg in hf_list]
    eeg_multisub = np.concatenate(tuple(eeg_list), axis=2)
    eog_multisub = np.concatenate(tuple(eog_list), axis=2)
    hf_multisub = np.concatenate(tuple(hf_list), axis=2)
    times = np.array(range(T))/fs
    return feats, eeg_multisub, eog_multisub, hf_multisub, fs, times

In [ ]:
# %%capture
# subjects = ['AS', 'YY', 'CM', 'SUB1', 'IR', 'SUB2', 'YZ', 'WD', 'CC', 'CW', 'WS', 'VC','HV','JC','DV','CD','JV','KY','KB','SC']
# bads = [[], ['B25'], ['A24'], ['B25'], ['B25'], ['B25'], ['A11','B7','B25'], ['B25','B31'], ['B25'], [], ['A1','A30','B25'], ['B25','B31'], ['B25','B31','A20','A21','A26','A31'], ['B25','B31','B32','A28','A29','A30'],['A25','A30','B25','B29'],['A30','B25','B31'],['A30','B25'],[],['B25'],['B25']]
# eeg_path_folder = "../../Experiments/data/CD/"
# video_ids = [dataset[0:2] for dataset in os.listdir(eeg_path_folder) if dataset.endswith('.set')]
# video_ids.remove('Mr')
# features_list = []
# eeg_multisub_list = []
# eog_multisub_list = []
# hf_multisub_list = []
# for video_id in video_ids:
#     features, eeg_multisub, eog_multisub, hf_multisub, fs, _ = multisub_data_org(subjects, video_id, fsStim=30, bads=bads, band=[15,25], eog=True, regression=True, normalize=True, smooth=True)
#     features_list.append(features) 
#     eeg_multisub_list.append(np.float32(eeg_multisub))
#     eog_multisub_list.append(np.float32(eog_multisub))
#     hf_multisub_list.append(np.float32(hf_multisub))
# if not os.path.exists('data/OneShot'):
#     os.makedirs('data/OneShot')
# with open('data/OneShot/features.pkl', 'wb') as f:
#     pickle.dump(features_list, f)
# with open('data/OneShot/eeg_multisub.pkl', 'wb') as f:
#     pickle.dump(eeg_multisub_list, f)
# with open('data/OneShot/eog_multisub.pkl', 'wb') as f:
#     pickle.dump(eog_multisub_list, f)
# with open('data/OneShot/hf_multisub.pkl', 'wb') as f:
#     pickle.dump(hf_multisub_list, f)

In [ ]:
subjects = ['AS', 'YY', 'CM', 'SUB1', 'IR', 'SUB2', 'YZ', 'WD', 'CC', 'CW', 'WS', 'VC','HV','JC','DV','CD','JV','KY','KB','SC']
fs = 30
with open('data/OneShot/features.pkl', 'rb') as f:
    features_list = pickle.load(f)
with open('data/OneShot/eeg_multisub.pkl', 'rb') as f:
    eeg_multisub_list = pickle.load(f)
with open('data/OneShot/eog_multisub.pkl', 'rb') as f:
    eog_multisub_list = pickle.load(f)
with open('data/OneShot/hf_multisub.pkl', 'rb') as f:
    hf_multisub_list = pickle.load(f)

In [ ]:
T = sum([F.shape[0] for F in features_list])
times = np.array(range(T))/fs
n_sub = len(subjects)
T/fs/60

In [ ]:
bio32 = mne.channels.make_standard_montage('biosemi32')
bio64 = mne.channels.make_standard_montage('biosemi64')
downsample_ind = [ch in bio32.ch_names for ch in bio64.ch_names]
# complem_ind = [ch not in bio32.ch_names for ch in bio64.ch_names]

In [ ]:
eeg_32_list = [eeg64[:,downsample_ind,:] for eeg64 in eeg_multisub_list]
hf_32_list = [hf64[:,downsample_ind,:] for hf64 in hf_multisub_list]
mix_64_list = [np.concatenate((eeg32, hf32), axis=1) for eeg32, hf32 in zip(eeg_32_list, hf_32_list)]

In [ ]:
avgflow_list = [feats[:,20] for feats in features_list]
avgtempctr_list = [feats[:,21] for feats in features_list]
objflow_list = [feats[:,8] for feats in features_list]
objtempctr_list = [feats[:,17] for feats in features_list]

## CCA

In [ ]:
def pip_CCA(eeg_multisub_list, feature_list, fs, n_sub, fig_name, tab_name, L_EEG=3, L_Stim=int(fs/2), offset_EEG=1, offset_Stim=0, signifi_level=True, n_components=5, ifEEG=True):
    cc = np.zeros((n_sub, n_components+3))
    for id_sub in range(n_sub):
        print('subject: ', id_sub+1)
        eeg_onesub_list = [eeg[:,:,id_sub] for eeg in eeg_multisub_list]
        CCA = algo.CanonicalCorrelationAnalysis(eeg_onesub_list, feature_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, signifi_level=signifi_level, n_components=n_components)
        _, corr_test, sig_corr, _, tsc_test, _, _, V_A_train, V_B_train = CCA.cross_val()
        cc[id_sub,0] = int(id_sub+1)
        cc[id_sub,1:n_components+1] = np.average(corr_test, axis=0)
        cc[id_sub,-2] = np.average(tsc_test)
        cc[id_sub,-1] = sig_corr
        eeg_onesub = np.concatenate(tuple(eeg_onesub_list), axis=0)
        forward_model = CCA.forward_model(eeg_onesub, V_A_train)
        if ifEEG:
            if not os.path.exists('figures/CCA/OneShot'):
                os.makedirs('figures/CCA/OneShot')
            utils.plot_spatial_resp(forward_model, corr_test, 'figures/CCA/OneShot/'+fig_name+str(id_sub+1)+'.png')
    columns = ['ID'] + ['CC'+str(i+1) for i in range(n_components)] + ['TSC(top2)','Sig_corr']
    df_cca = pd.DataFrame(cc, columns = columns)
    if not os.path.exists('tables/CCA/OneShot'):
        os.makedirs('tables/CCA/OneShot')
    df_cca.to_csv('tables/CCA/OneShot/'+tab_name+'.csv', index=False)

In [ ]:
pip_CCA(eeg_multisub_list, avgflow_list, fs, n_sub, 'SR_avgflow_', 'avgflow_eeg')

In [ ]:
pip_CCA(eeg_multisub_list, avgtempctr_list, fs, n_sub, 'SR_avgtempctr_', 'avgtempctr_eeg')

In [ ]:
pip_CCA(eeg_multisub_list, objflow_list, fs, n_sub, 'SR_objflow_', 'objflow_eeg')

In [ ]:
pip_CCA(eeg_multisub_list, objtempctr_list, fs, n_sub, 'SR_objtempctr_', 'objtempctr_eeg')

In [ ]:
# Note: there are 4 EOG channels, so the number of components should be less than 4. Or include time lags in EOG and set n_components higher.
pip_CCA(eog_multisub_list, objflow_list, fs, n_sub, 'DoesNotMatter', 'objflow_eog', ifEEG=False)

In [ ]:
pip_CCA(eog_multisub_list, objtempctr_list, fs, n_sub, 'DoesNotMatter', 'objtempctr_eog', ifEEG=False)

In [ ]:
# To include high frequency information. Turns out that it does not help.
# pip_CCA(mix_64_list, avgflow_list, fs, n_sub, 'DoesNotMatter', 'avgflow_mixeeg', ifEEG=False)

### Match-Mismatch Setting

In [ ]:
def pip_mm(eeg_multisub_list, feature_list, fs, n_sub, len_trial, tab_name, component, L_EEG=3, L_Stim=int(fs/2), offset_EEG=1, offset_Stim=0, signifi_level=False, n_components=5):
    err = np.zeros((n_sub, 3))
    for id_sub in range(n_sub):
        eeg_onesub_list = [eeg[:,:,id_sub] for eeg in eeg_multisub_list]
        CCA = algo.CanonicalCorrelationAnalysis(eeg_onesub_list, feature_list, fs, L_EEG, L_Stim, offset_EEG, offset_Stim, signifi_level=signifi_level, n_components=n_components)
        corr_tensor_list, tsc_mtx_list = CCA.match_mismatch(len_trial)
        if component is not None:
            match_stim_err, match_eeg_err = utils.eval_mm(corr_tensor_list, component)
        else:
            match_stim_err, match_eeg_err = utils.eval_mm(tsc_mtx_list, component=None)
        err[id_sub,0] = int(id_sub+1)
        err[id_sub,1] = match_stim_err
        err[id_sub,2] = match_eeg_err
    columns = ['ID', 'Match_Stim_err', 'Match_EEG_err']
    df_cca = pd.DataFrame(err, columns = columns)
    if not os.path.exists('tables/CCA/MM/OneShot'):
        os.makedirs('tables/CCA/MM/OneShot')
    df_cca.to_csv('tables/CCA/MM/OneShot/'+tab_name+'.csv', index=False)

In [ ]:
pip_mm(eeg_multisub_list, avgflow_list, fs, n_sub, 60, 'avgflow_1m_corr_1', 1)
pip_mm(eeg_multisub_list, objflow_list, fs, n_sub, 60, 'objflow_1m_corr_1', 1)
pip_mm(eeg_multisub_list, avgtempctr_list, fs, n_sub, 60, 'avgtempctr_1m_corr_1', 1)
pip_mm(eeg_multisub_list, objtempctr_list, fs, n_sub, 60, 'objtempctr_1m_corr_1', 1)
pip_mm(eeg_multisub_list, avgflow_list, fs, n_sub, 60, 'avgflow_1m_tsc', None)
pip_mm(eeg_multisub_list, objflow_list, fs, n_sub, 60, 'objflow_1m_tsc', None)
pip_mm(eeg_multisub_list, avgtempctr_list, fs, n_sub, 60, 'avgtempctr_1m_tsc', None)
pip_mm(eeg_multisub_list, objtempctr_list, fs, n_sub, 60, 'objtempctr_1m_tsc', None)

## GCCA

In [ ]:
def pip_GCCA(eeg_multisub_list, feature_list, fs, L, offset, name_app, regress_out=False, shifted_ver=True, fold=10, n_components=10, n_MC=100, min_shift_s=10):
    n_sub = eeg_multisub_list[0].shape[2]
    eeg_hankel_list = [utils.hankelize_eeg_multisub(eeg, L=L, offset=offset) for eeg in eeg_multisub_list]
    if regress_out:
        feature_hankel_list = [utils.block_Hankel(feat, L=int(fs/2)) for feat in feature_list]
        eeg_multisub_reg_list = [utils.regress_out(eeg, feat) for eeg, feat in zip(eeg_hankel_list, feature_hankel_list)]
    else:
        eeg_multisub_reg_list = eeg_hankel_list
    GCCA = algo.GeneralizedCCA(eeg_multisub_reg_list, fs=fs, L=1, fold=fold, offset=0, n_components=n_components, signifi_level=True, trials=False, save_W_perfold=True)
    _, corr_test_reg, _, cov_test_reg, _, tsc_test_reg, _, _, _, F_train_reg = GCCA.cross_val()
    W_train_list = GCCA.W_train_list
    test_list = GCCA.test_list
    F_train_reg = utils.F_organize(F_train_reg, L=5, offset=2, avg=False)
    print('tsc_test: ', np.mean(tsc_test_reg, axis=0))
    print('Average ISCov of the top 10 components on the test sets: ', np.mean(cov_test_reg, axis=0))
    utils.plot_spatial_resp(F_train_reg, corr_test_reg, 'figures/GCCA/OneShot/'+str(n_sub)+'_'+name_app+'.pdf', ifISC=True)
    if shifted_ver:
        ISCOV_rand = np.zeros((n_MC*fold, n_components))
        for i in range(fold):
            W_train = W_train_list[i]
            test = test_list[i]
            for count in tqdm(range(n_MC)):
                test_shift, offset_list = utils.random_shift_3D(test, min_shift_s*fs)
                _, ISCOV_rand[i*n_MC+count,:], _, _ = GCCA.avg_stats(test_shift, W_train)
        print('Mean of the estimate error due to finite sample size: ', np.mean(ISCOV_rand))
        print('Standard deviation of the estimate error due to finite sample size: ', np.std(ISCOV_rand))
    return corr_test_reg, cov_test_reg, ISCOV_rand, W_train_list, test_list

In [ ]:
GCCA = algo.GeneralizedCCA(eeg_multisub_list, fs, L=5, offset=2, n_components=10, signifi_level=True, trials=False)
corr_train, corr_test, cov_train, cov_test, tsc_train, tsc_test, dist_train, dist_test, W_train, F_train_GCCA = GCCA.cross_val()
print('tsc_test: ', np.mean(tsc_test, axis=0))
print('Average ISCov of the top 10 components on the test sets: ', np.mean(cov_test, axis=0))
if not os.path.exists('figures/GCCA/OneShot'):
    os.makedirs('figures/GCCA/OneShot')
utils.plot_spatial_resp(F_train_GCCA, corr_test, 'figures/GCCA/OneShot/'+str(n_sub)+'_cb.pdf', ifISC=True)

In [ ]:
# Just for fair comparison: Hankelize first and then apply GCCA
# The results are slightly different from the results obtained above
eeg_hankel_list = [utils.hankelize_eeg_multisub(eeg, L=5, offset=2) for eeg in eeg_multisub_list]
GCCA = algo.GeneralizedCCA(eeg_hankel_list, fs=30, L=1, offset=0, n_components=10, signifi_level=True, trials=False)
_, corr_test, _, cov_test, _, _, _, _, _, F_train = GCCA.cross_val()
print('tsc_test: ', np.mean(tsc_test, axis=0))
print('Average ISCov of the top 10 components on the test sets: ', np.mean(cov_test, axis=0))

In [ ]:
# Just for fair comparison: Hankelize first and then apply GCCA
ISC_of, ISCOV_of, err_of, W_train_of, test_of = pip_GCCA(eeg_multisub_list, objflow_list, fs, L=5, offset=2, name_app='OG_cb', regress_out=False)

In [ ]:
# regress out ObjFlow
ISC_of, ISCOV_of, err_of, W_train_of, test_of = pip_GCCA(eeg_multisub_list, objflow_list, fs, L=5, offset=2, name_app='regOF_cb', regress_out=True)

In [ ]:
# regress out ObjTempCtr
ISC_otc, ISCOV_otc, err_otc, W_train_otc, test_otc = pip_GCCA(eeg_multisub_list, objtempctr_list, fs, L=5, offset=2, name_app='regOTC_cb', regress_out=True)

### Channel selection

In [ ]:
def greedy_selection(eeg_multisub_list, fs, forward_model=None, kth_MC=1, L=5, offset=2, n_components=10, comp=1, signifi_level=False, trials=False, max_ch=None):
    nb_channel = eeg_multisub_list[0].shape[1]
    if max_ch is None:
        max_ch = nb_channel
    cc = np.zeros((max_ch, n_components+2))
    columns = ['#channel'] + ['CC'+str(i+1) for i in range(n_components)] + ['TSC(top4)']
    if not os.path.exists('tables/GCCA/ChannelSelection'):
        os.makedirs('tables/GCCA/ChannelSelection')
    if forward_model is not None:
        fm_comp = forward_model[:,comp-1]
        rank_idx = np.argsort(np.abs(fm_comp))[::-1]
        file_name = 'GreedyComp_' + str(comp) + '.csv'
    else:
        rank_idx = np.random.permutation(nb_channel)
        file_name = 'RandSele_' + str(kth_MC) + '.csv'
    for i in range(1, max_ch+1):
        print('Selected Channels: ', rank_idx[:i])
        eeg_channel = [eeg[:,rank_idx[:i],:] for eeg in eeg_multisub_list]
        GCCA = algo.GeneralizedCCA(eeg_channel, fs, L=L, offset=offset, n_components=n_components, signifi_level=signifi_level, trials=trials)
        _, corr_test, _, _, _, tsc_test, _, _, _, _ = GCCA.cross_val()
        cc[i-1,0] = i
        cc[i-1,1:n_components+1] = np.average(corr_test, axis=0)
        cc[i-1,-1] = np.average(tsc_test)
        print('tsc_test: ', np.average(tsc_test))
        df_cca = pd.DataFrame(cc, columns = columns)
        df_cca.to_csv('tables/GCCA/ChannelSelection/'+file_name, index=False)
    ISC_1st = cc[:,1]
    return rank_idx, ISC_1st

In [ ]:
rank_idx, _ = greedy_selection(eeg_multisub_list, fs, forward_model=F_train_GCCA)

In [ ]:
# # Takes long time to run
nb_MC = 30
for kth_MC in range(nb_MC):
    # randomly generate a channel order
    _, _ = greedy_selection(eeg_multisub_list, fs, forward_model=None, kth_MC=kth_MC)

In [ ]:
# load the saved csv file
gs_cca = pd.read_csv('tables/GCCA/ChannelSelection/comp_1.csv')
# load the csv files that contain 'Rand' in their names
rand_csv_file = [name for name in os.listdir('tables/GCCA/ChannelSelection/') if 'Rand' in name]
rand_cca_list = []
for i in range(len(rand_csv_file)):
    df = pd.read_csv('tables/GCCA/ChannelSelection/' + rand_csv_file[i])
    if df['CC1'].iloc[-1] != 0:
        rand_cca_list.append(df)
# merge the 'CC1' columns of all the csv files into one dataframe
rand_CC1 = pd.concat([rand_cca_list[i]['CC1'] for i in range(len(rand_cca_list))], axis=1)
# get the 5-50-95 percentile of the CC1 values
rand_CC1_percentile = rand_CC1.quantile([0.05, 0.5, 0.95], axis=1).T

In [ ]:
plt.close()
plt.figure(figsize=(6, 4))
plt.plot(gs_cca['#channel'], gs_cca['CC1'], label='Greedy Selection')
# plot the 5-50-95 percentile of the CC1 values
plt.fill_between(gs_cca['#channel'], rand_CC1_percentile[0.05], rand_CC1_percentile[0.95], alpha=0.2)
plt.plot(gs_cca['#channel'], rand_CC1_percentile[0.5], linestyle='--', color='k', label='Random Selection (median)')
plt.xlabel('Number of channels')
plt.ylabel('ISC (1st component)')
plt.xlim([1, 64])
plt.legend()
plt.savefig('../../Manuscript/1st/images/channel_selection.jpeg', dpi=600)
plt.show()

## GCCA + LS
This section is not included in the paper.

In [ ]:
LS_GCCA_Group = algo.LSGCCA_Group(eeg_multisub_list, objflow_list, fs, L_EEG=5, L_Stim=int(fs/2), offset_EEG=2, offset_Stim=0, id_sub=0, n_components=10)
LS_GCCA_Group.to_latent_space()
corr_train, corr_test, sig_corr, We_train, Ws_train, F_train = LS_GCCA_Group.cross_val()

In [ ]:
plt.close()
# n_comp = Ws_train.shape[1]
for i in range(5):
    w, h = signal.freqz(Ws_train[:,i])
    freq = w / (2*np.pi) * fs
    plt.plot(freq, 20*np.log10(abs(h)), label='Component {}'.format(i+1))
plt.legend()
plt.xlabel('Frequency [Hz]')
plt.ylabel('Magnitude [dB]')
plt.title('Frequency response of the filters for ObjFlow')
plt.show()

In [ ]:
LS_GCCA_Group = algo.LSGCCA_Group(eeg_multisub_list, avgflow_list, fs, L_EEG=5, L_Stim=int(fs/2), offset_EEG=2, offset_Stim=0, id_sub=0, n_components=10)
LS_GCCA_Group.to_latent_space()
corr_train, corr_test, sig_corr, We_train, Ws_train, F_train = LS_GCCA_Group.cross_val()

## Results Visualization

In [ ]:
import textwrap

def df_split_two_comp(filename):
    df = pd.read_csv(filename)
    two_comp = df[['CC1', 'CC2']]
    CC1_high = df[df['CC1']>df['Sig_corr']]['CC1']
    CC1_low = df[df['CC1']<df['Sig_corr']]['CC1']
    CC2_high = df[df['CC2']>df['Sig_corr']]['CC2']
    CC2_low = df[df['CC2']<df['Sig_corr']]['CC2']
    avg_sig_corr = df['Sig_corr'].mean()
    high_two_comp = pd.concat([pd.DataFrame(CC1_high), pd.DataFrame(CC2_high)])
    low_two_comp = pd.concat([pd.DataFrame(CC1_low), pd.DataFrame(CC2_low)])
    return two_comp, high_two_comp, low_two_comp, avg_sig_corr

def plot_data(ax, data, high, low, label, avg_sig_corr):
    sns.pointplot(data=data, errorbar='sd', join=False, estimator='mean', ax=ax)
    sns.swarmplot(data=high, palette=['0', '0'], alpha=.35, ax=ax)
    if not low.isnull().values.all():
        sns.swarmplot(data=low, palette=['red', 'red'], alpha=.75, ax=ax)
    # draw line for significant correlation, set the transparency of the line to 0.5
    ax.axhline(avg_sig_corr, ls='--', color='black', alpha=.5)
    # set xticks
    ax.set_xticks(['CC1', 'CC2'])
    wrapped_title = textwrap.fill(label, 15)
    ax.set_title(wrapped_title)

def visual_two_comp(files, labels, fig_name):
    plt.close()
    nb_feats = len(files)
    fig, axs = plt.subplots(1, nb_feats, sharey=True, figsize=(int(nb_feats*2), 4))
    axs[0].set_ylabel('Correlation')
    for i in range(nb_feats):
        data, high, low, avg_sig_corr = df_split_two_comp(files[i])
        plot_data(axs[i], data, high, low, labels[i], avg_sig_corr)
    plt.savefig('../../Manuscript/1st/images/' + fig_name + '.jpeg', dpi=600)
    plt.show()


In [ ]:
files = ['tables/CCA/MrBean/avgflow_eeg.csv', 'tables/CCA/MrBean/avgflow_clean_eeg.csv', 
         'tables/CCA/MrBean/avgtempctr_eeg.csv', 'tables/CCA/MrBean/avgtempctr_clean_eeg.csv', 
         'tables/CCA/MrBean/binary_eeg.csv']
labels = ['AvgFlow With Shot Cuts', 'AvgFlow Without Shot Cuts', 'AvgTempCtr With Shot Cuts', 'AvgTempCtr Without Shot Cuts', 'Binary Shot Cut']
visual_two_comp(files, labels, 'MrBean_two_comp')

In [ ]:
files = ['tables/CCA/OneShot/avgflow_eeg.csv', 'tables/CCA/OneShot/objflow_eeg.csv', 
         'tables/CCA/OneShot/avgtempctr_eeg.csv', 'tables/CCA/OneShot/objtempctr_eeg.csv']
labels = ['AvgFlow', 'ObjFlow', 'AvgTempCtr', 'ObjTempCtr']
visual_two_comp(files, labels, 'OneShot_two_comp')

In [ ]:
def read_and_select(filename, column):
    df = pd.read_csv(filename)
    return df.loc[:, column]

def plot_df(files, labels, col, y_name, fig_name):
    plt.close()
    nb_cols = len(labels)
    df_concat = pd.DataFrame({labels[i]: read_and_select(files[i], col) for i in range(nb_cols)})
    sns.pointplot(data=df_concat, errorbar='sd', join=False, estimator='mean')
    sns.swarmplot(data=df_concat, color="0", alpha=.35)
    plt.ylabel(y_name)
    plt.savefig('../../Manuscript/1st/images/' + fig_name + '.jpeg', dpi=600)
    plt.show()

In [ ]:
files = ['tables/CCA/OneShot/avgflow_eeg.csv', 'tables/CCA/OneShot/avgtempctr_eeg.csv', 
         'tables/CCA/OneShot/objflow_eeg.csv', 'tables/CCA/OneShot/objtempctr_eeg.csv']
labels = ['AvgFlow-EEG', 'AvgTempCtr-EEG', 'ObjFlow-EEG', 'ObjTempCtr-EEG']
col = 'TSC(top2)'
plot_df(files, labels, col, 'TSC (top 2)', 'avg_obj_tsc')

In [ ]:
files = ['tables/CCA/OneShot/objflow_eog.csv', 'tables/CCA/OneShot/objflow_eeg.csv', 
         'tables/CCA/OneShot/objtempctr_eog.csv', 'tables/CCA/OneShot/objtempctr_eeg.csv']
labels = ['ObjFlow-EOG', 'ObjFlow-EEG', 'ObjTempCtr-EOG', 'ObjTempCtr-EEG']
col = 'TSC(top2)'
plot_df(files, labels, col, 'TSC (top 2)', 'eeg_eog_tsc')

In [ ]:
files = ['tables/CCA/MM/OneShot/avgflow_1m_corr_1.csv', 'tables/CCA/MM/OneShot/avgtempctr_1m_corr_1.csv', 
         'tables/CCA/MM/OneShot/objflow_1m_corr_1.csv', 'tables/CCA/MM/OneShot/objtempctr_1m_corr_1.csv']
labels = ['AvgFlow', 'AvgTempCtr', 'ObjFlow', 'ObjTempCtr']
col = 'Match_EEG_err'
plot_df(files, labels, col, 'Error Rate', 'Match_EEG_err')

In [ ]:
files = ['tables/CCA/MM/OneShot/avgflow_1m_corr_1.csv', 'tables/CCA/MM/OneShot/avgtempctr_1m_corr_1.csv', 
         'tables/CCA/MM/OneShot/objflow_1m_corr_1.csv', 'tables/CCA/MM/OneShot/objtempctr_1m_corr_1.csv']
labels = ['AvgFlow', 'AvgTempCtr', 'ObjFlow', 'ObjTempCtr']
col = 'Match_Stim_err'
plot_df(files, labels, col, 'Error Rate', 'Match_Stim_err')

In [ ]:
def read_and_select(filename, col_name='TSC(top2)'):
    df = pd.read_csv(filename)
    return df[col_name]

def plot_data(ax, data, label):
    sns.pointplot(data=data, errorbar='sd', join=False, estimator='mean', ax=ax)
    sns.swarmplot(data=data, palette=['0', '0'], alpha=.35, ax=ax)
    ax.set_xticks(['EOG', 'EEG'])
    ax.set_title(label)

def visual_EEG_EOG(files_flow, files_tempctr, features, fig_name):
    plt.close()
    nb_feats = len(features)
    df_flow = pd.DataFrame({'EOG': read_and_select(files_flow[0]), 'EEG': read_and_select(files_flow[1])})
    df_tempctr = pd.DataFrame({'EOG': read_and_select(files_tempctr[0]), 'EEG': read_and_select(files_tempctr[1])})
    fig, axs = plt.subplots(1, nb_feats, sharey=True, figsize=(int(3*nb_feats), 4))
    axs[0].set_ylabel('TSC (top 2)')
    plot_data(axs[0], df_flow, features[0])
    plot_data(axs[1], df_tempctr, features[1])
    plt.savefig('../../Manuscript/1st/images/' + fig_name + '.jpeg', dpi=600)
    plt.show()


In [ ]:
files_flow = ['tables/CCA/OneShot/objflow_eog.csv', 'tables/CCA/OneShot/objflow_eeg.csv']
files_tempctr = ['tables/CCA/OneShot/objtempctr_eog.csv', 'tables/CCA/OneShot/objtempctr_eeg.csv']
features = ['ObjFlow', 'ObjTempCtr']
visual_EEG_EOG(files_flow, files_tempctr, features, 'eeg_eog_tsc')